In [1]:
import sys  
sys.path.insert(0, '..')

In [2]:
import jax

In [3]:
jax.device_count()

8

In [4]:
# requirements
from sklearn.model_selection import train_test_split
import gzip
from tqdm import tqdm
import numpy as np
import csv

# for training script
from dataclasses import dataclass, field, asdict, replace
from functools import partial
from typing import Callable, List, Union

import jax
import jax.numpy as jnp
import optax
from flax import jax_utils, struct, traverse_util
from flax.training import train_state
from flax.serialization import to_bytes, from_bytes
from flax.training.common_utils import shard
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast
from trainer.loss.custom import multiple_negatives_ranking_loss


import wandb
import json
import os

from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, FlaxAutoModel
import datasets


from trainer.utils.ops import normalize_L2, mean_pooling



In [5]:
@dataclass
class TrainingArgs:
    model_id: str = "microsoft/codebert-base"
    max_epochs: int = 2
    batch_size_per_device: int = 8
    seed: int = 42
    lr: float = 2e-5
    init_lr: float = 1e-5
    warmup_steps: int = 2000
    weight_decay: float = 1e-3

    input1_maxlen: int = 128
    input2_maxlen: int = 128
    
    logging_steps: int = 20
    save_dir: str = "checkpoints"

    tr_data_files: List[str] = field(
        default_factory=lambda: [
            "tr.csv",
        ]
    )
        
    val_data_files: List[str] = field(
        default_factory=lambda: [
            "val.csv",
        ]
    )

    def __post_init__(self):
        self.batch_size = self.batch_size_per_device * jax.device_count()


In [6]:


def scheduler_fn(lr, init_lr, warmup_steps, num_train_steps):
    decay_steps = num_train_steps - warmup_steps
    warmup_fn = optax.linear_schedule(init_value=init_lr, end_value=lr, transition_steps=warmup_steps)
    decay_fn = optax.linear_schedule(init_value=lr, end_value=1e-7, transition_steps=decay_steps)
    lr = optax.join_schedules(schedules=[warmup_fn, decay_fn], boundaries=[warmup_steps])
    return lr

def build_tx_0(lr, init_lr, warmup_steps, num_train_steps, weight_decay):
    def weight_decay_mask(params):
        params = traverse_util.flatten_dict(params)
        mask = {k: (v[-1] != "bias" and v[-2:] != ("LayerNorm", "scale")) for k, v in params.items()}
        return traverse_util.unflatten_dict(mask)
    lr = scheduler_fn(lr, init_lr, warmup_steps, num_train_steps)
    tx = optax.adamw(learning_rate=lr, weight_decay=weight_decay, mask=weight_decay_mask)
    return tx, lr

def warmup_and_constant(lr, init_lr, warmup_steps):
    warmup_fn = optax.linear_schedule(init_value=init_lr, end_value=lr, transition_steps=warmup_steps)
    constant_fn = optax.constant_schedule(value=lr)
    lr = optax.join_schedules(schedules=[warmup_fn, constant_fn], boundaries=[warmup_steps])
    return lr

def build_tx(lr, init_lr, warmup_steps, weight_decay):
    def weight_decay_mask(params):
        params = traverse_util.flatten_dict(params)
        mask = {k: (v[-1] != "bias" and v[-2:] != ("LayerNorm", "scale")) for k, v in params.items()}
        return traverse_util.unflatten_dict(mask)
    lr = warmup_and_constant(lr, init_lr, warmup_steps)
    tx = optax.adamw(learning_rate=lr, weight_decay=weight_decay, mask=weight_decay_mask)
    return tx, lr


In [7]:

class TrainState(train_state.TrainState):
    loss_fn: Callable = struct.field(pytree_node=False)
    scheduler_fn: Callable = struct.field(pytree_node=False)


@partial(jax.pmap, axis_name="batch")
def train_step(state, model_input1, model_input2, drp_rng):
    train = True
    new_drp_rng, drp_rng = jax.random.split(drp_rng, 2)

    def loss_fn(params, model_input1, model_input2, drp_rng):
        def _forward(model_input):
            attention_mask = model_input["attention_mask"]
            model_output = state.apply_fn(**model_input, params=params, train=train, dropout_rng=drp_rng)

            embedding = mean_pooling(model_output, attention_mask)
            embedding = normalize_L2(embedding)

            # gather all the embeddings on same device for calculation loss over global batch
            embedding = jax.lax.all_gather(embedding, axis_name="batch")
            embedding = jnp.reshape(embedding, (-1, embedding.shape[-1]))

            return embedding

        embedding1, embedding2 = _forward(model_input1), _forward(model_input2)
        return state.loss_fn(embedding1, embedding2)

    grad_fn = jax.value_and_grad(loss_fn)
    loss, grads = grad_fn(state.params, model_input1, model_input2, drp_rng)
    state = state.apply_gradients(grads=grads)

    step = jax.lax.pmean(state.step, axis_name="batch")
    metrics = {"train_loss": loss, "lr": state.scheduler_fn(step)}

    return state, metrics, new_drp_rng

In [19]:

@partial(jax.pmap, axis_name="batch")
def val_step(state, model_inputs1, model_inputs2):
    train = False

    def _forward(model_input):
        attention_mask = model_input["attention_mask"][..., None]
        embedding = state.apply_fn(**model_input, params=state.params, train=train)[0]
        attention_mask = jnp.broadcast_to(attention_mask, jnp.shape(embedding))

        embedding = embedding * attention_mask
        embedding = jnp.mean(embedding, axis=1)

        modulus = jnp.sum(jnp.square(embedding), axis=-1, keepdims=True)
        embedding = embedding / jnp.maximum(modulus, 1e-12)

        # gather all the embeddings on same device for calculation loss over global batch
        embedding = jax.lax.all_gather(embedding, axis_name="batch")
        embedding = jnp.reshape(embedding, (-1, embedding.shape[-1]))

        return embedding

    embedding1, embedding2 = _forward(model_inputs1), _forward(model_inputs2)
    loss = state.loss_fn(embedding1, embedding2)
    return jnp.mean(loss)



def get_batched_dataset(dataset, batch_size, seed=None):
    if seed is not None:
        dataset = dataset.shuffle(seed=seed)
    for i in range(len(dataset) // batch_size):
        batch = dataset[i*batch_size: (i+1)*batch_size]
        yield dict(batch)


@dataclass
class DataCollator:
    tokenizer: Union[PreTrainedTokenizerFast, PreTrainedTokenizer]
    input1_maxlen: int = 128
    input2_maxlen: int = 128

    def __call__(self, batch):
        # Currently only static padding; TODO: change below for adding dynamic padding support
        model_input1 = self.tokenizer(batch["docstring"], return_tensors="jax", max_length=self.input1_maxlen, truncation=True, padding="max_length")
        model_input2 = self.tokenizer(batch["code"], return_tensors="jax", max_length=self.input2_maxlen, truncation=True, padding="max_length")
        model_input1, model_input2 = dict(model_input1), dict(model_input2)
        return shard(model_input1), shard(model_input2)


def save_checkpoint(save_dir, state, save_fn=None, training_args=None):
    print(f"saving checkpoint in {save_dir}", end=" ... ")

    os.makedirs(save_dir, exist_ok=True)
    state = jax_utils.unreplicate(state)

    if save_fn is not None:
        # saving model in HF fashion
        save_fn(save_dir, params=state.params)
    else:
        path = os.path.join(save_dir, "flax_model.msgpack")
        with open(path, "wb") as f:
            f.write(to_bytes(state.params))

    # this will save optimizer states
    path = os.path.join(save_dir, "opt_state.msgpack")
    with open(path, "wb") as f:
        f.write(to_bytes(state.opt_state))

    if training_args is not None:
        path = os.path.join(save_dir, "training_args.json")
        with open(path, "w") as f:
            json.dump(asdict(training_args), f)

    print("done!!")


def prepare_dataset(args):
    # tr_dataset = load_dataset("csv", data_files=args.tr_data_files, split="train")
    # val_dataset = load_dataset("csv", data_files=args.val_data_files, split="val")
    dataset = datasets.load_dataset("csv", data_files={
      "train": "../data/codesearchnet_train.csv.gz",
      "test": "../data/codesearchnet_test.csv.gz",
      "validation": "../data/codesearchnet_validation.csv.gz"
    })

    # columns_to_remove = ['repo', 'path', 'func_name', 'original_string', 'sha', 'url', 'partition']
    # dataset = dataset.remove_columns(columns_to_remove)

    # drop extra batch from the end
    for split in dataset:
        num_samples = len(dataset[split]) - len(dataset[split]) % args.batch_size
        dataset[split] = dataset[split].shuffle(seed=args.seed).select(range(num_samples))

    tr_dataset, val_dataset = dataset["train"], dataset["validation"]
    return tr_dataset, val_dataset



In [9]:
args = TrainingArgs()

logger = wandb.init(project="code-search-net", config=asdict(args))

logging_dict = dict(logger.config); logging_dict["save_dir"] += f"-{logger.id}"
args = replace(args, **logging_dict)

print(args)
#main(args, None)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mandubian (use `wandb login --relogin` to force relogin)


TrainingArgs(model_id='microsoft/codebert-base', max_epochs=2, batch_size_per_device=8, seed=42, lr=2e-05, init_lr=1e-05, warmup_steps=2000, weight_decay=0.001, input1_maxlen=128, input2_maxlen=128, logging_steps=20, save_dir='checkpoints-2keyqiu9', tr_data_files=['tr.csv'], val_data_files=['val.csv'])


In [10]:
model = FlaxAutoModel.from_pretrained(args.model_id)

Downloading:   0%|          | 0.00/498 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [11]:
tokenizer = AutoTokenizer.from_pretrained(args.model_id)

Downloading:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [12]:
data_collator = DataCollator(
    tokenizer=tokenizer,
    input1_maxlen=args.input1_maxlen,
    input2_maxlen=args.input2_maxlen,
)


In [17]:
!ls -la

total 16
drwxrwxr-x  3 pascal_voitot pascal_voitot 4096 Jul  7 12:48 .
drwxrwxr-x 17 pascal_voitot pascal_voitot 4096 Jul  7 12:39 ..
-rw-rw-r--  1 pascal_voitot pascal_voitot  875 Jul  7 12:10 codesearch.ipynb
drwxrwxr-x  4 pascal_voitot pascal_voitot 4096 Jul  7 12:51 wandb


In [20]:

tr_dataset, val_dataset = prepare_dataset(args)


0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/pascal_voitot/.cache/huggingface/datasets/csv/default-83af4b22d0d2ded6/0.0.0/e138af468cb14e747fb46a19c787ffcfa5170c821476d20d5304287ce12bbc23. Subsequent calls will reuse this data.


In [21]:
tx_args = {
    "lr": args.lr,
    "init_lr": args.init_lr,
    "warmup_steps": args.warmup_steps,
    "weight_decay": args.weight_decay,
}
tx, lr = build_tx(**tx_args)

In [22]:

state = TrainState.create(
    apply_fn=model.__call__,
    params=model.params,
    tx=tx,
    loss_fn=multiple_negatives_ranking_loss,
    scheduler_fn=lr,
)
state = jax_utils.replicate(state)

rng = jax.random.PRNGKey(args.seed)
drp_rng = jax.random.split(rng, jax.device_count())


/home/pascal_voitot/.cache/pypoetry/virtualenvs/flax-sentence-embeddings-zVHyP_u9-py3.9/lib/python3.9/site-packages/jax/lib/xla_bridge.py:382: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(
/home/pascal_voitot/.cache/pypoetry/virtualenvs/flax-sentence-embeddings-zVHyP_u9-py3.9/lib/python3.9/site-packages/jax/lib/xla_bridge.py:369: UserWarning: jax.host_id has been renamed to jax.process_index. This alias will eventually be removed; please update your code.
  warnings.warn(


In [26]:
for epoch in range(args.max_epochs):
    # training step
    total = len(tr_dataset) // args.batch_size
    batch_iterator = get_batched_dataset(tr_dataset, args.batch_size, seed=epoch)
    for i, batch in tqdm(enumerate(batch_iterator), desc=f"Running epoch-{epoch}", total=total):
        model_input1, model_input2 = data_collator(batch)
        state, metrics, drp_rng = train_step(state, model_input1, model_input2, drp_rng)

        #print("metrics", metrics)
        if (i + 1) % args.logging_steps == 0:
            train_loss = jax_utils.unreplicate(metrics["train_loss"]).item()
            # tqdm.write(str(dict(train_loss=train_loss, step=i+1)))
            logger.log({
                "train_loss": train_loss,
                "step": i + 1,
            }, commit=True)

    # evaluation
    val_loss  = jnp.array(0.)
    total = len(val_dataset) // args.batch_size
    val_batch_iterator = get_batched_dataset(val_dataset, args.batch_size, seed=None)
    for j, batch in tqdm(enumerate(val_batch_iterator), desc=f"Eval after epoch-{epoch}", total=total):
       model_input1, model_input2 = data_collator(batch)
       val_step_loss = val_step(state, model_input1, model_input2)
       val_loss += jax_utils.unreplicate(val_step_loss)

    val_loss = val_loss.item() / (j + 1)
    logger.log({"val_loss": val_loss}, commit=True)
    
    save_dir = args.save_dir + f"-epoch-{epoch}"
    save_checkpoint(save_dir, state, save_fn=model.save_pretrained, training_args=args)            


Running epoch-0:   0%|          | 0/10657 [00:00<?, ?it/s]

{'train_loss': 0.07115189731121063, 'step': 20}
{'train_loss': 0.12793439626693726, 'step': 40}
{'train_loss': 0.026729723438620567, 'step': 60}
{'train_loss': 0.11590759456157684, 'step': 80}
{'train_loss': 0.030272899195551872, 'step': 100}
{'train_loss': 0.028267743065953255, 'step': 120}
{'train_loss': 0.023153021931648254, 'step': 140}


KeyboardInterrupt: 